In [504]:
import pandas as pd
import numpy as np
import operator

In [523]:
training = pd.read_csv('train.data')
training = training.drop(columns='animal_name')

In [524]:
training

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,classtype
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,0,0,1,0,1,0,0,0,0,1,0,0,6,0,0,0,6
4,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
5,1,0,1,0,1,0,0,0,0,1,0,0,6,0,0,0,6
6,0,1,1,0,1,1,1,0,1,1,0,0,2,1,0,0,2
7,0,0,1,0,0,1,1,0,0,0,0,0,4,0,0,0,7
8,1,0,0,1,1,0,0,1,1,1,0,0,2,1,0,0,1
9,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,0,3


In [525]:
test = pd.read_csv('test.data')
test = test.drop(columns=['animal_name', 'classtype'])
test = test.values.tolist()

In [526]:
training.values

array([[1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 7],
       [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 4],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 6, 0, 0, 0, 6],
       [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 1, 0, 1, 1],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 6, 0, 0, 0, 6],
       [0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2],
       [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 7],
       [1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3]])

In [527]:
col_names = training.columns.tolist()

In [528]:
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

In [529]:
def prior_proba(column_values):
    prior_dict = {}
    length = len(column_values)
    for val in column_values:
        if val not in prior_dict.keys():
            prior_dict[val] = 1
        else:
            prior_dict[val] += 1
    
    for key, val in prior_dict.items():
        prior_dict[key] = val / length;
    return prior_dict

In [530]:
def get_column_values(matrix, i):
    return [row[i] for row in matrix]

In [538]:
def likelihood_proba(train, test, col_names):
    labels = get_column_values(train.values.tolist(), -1)
    unique_Labels = list(set(labels))
    separated = separate_by_class(train.values.tolist())
    likelihood_prob = {}
    
    for i in unique_Labels:
        class_list = []
        for j in range(0, len(test)):
            num = 0;
            for c_i in range(len(separated[i])):
                if separated[i][c_i][j] == test[j]:
                    num +=1
            num += 0.1
                    
            den = len(separated[i]) + 0.1
            # Laplace Correction : 
#             if num == 0.0:
#                 num = 0.1
#                 den = den + len(np.unique(data[col_nm[j]])) + 0.1
            val = num / den
            class_list.append(val)
        likelihood_prob[i] = class_list
    return likelihood_prob

In [539]:
def posterior_proba(data, X_list):
    prior_prob = {}
    likelihood_prob = {}
    
    prior_prob = prior_proba(data.iloc[:, -1].values)
    
    likelihood_prob = likelihood_proba(data, X_list, col_names)
    
    posterior_prob = {}
    for i in prior_prob:
        n1 = prior_prob[i]
        n2 = likelihood_prob[i]
        for j in range(0, len(n2)):
            n1 = n1 * n2[j]
        posterior_prob[i] = n1
    return posterior_prob

In [540]:
def get_label(posterior_prob):
    return max(posterior_prob.items(), key=operator.itemgetter(1))

In [541]:
posterior_prob = posterior_proba(training, test[0])

In [542]:
posterior_prob

{1: 6.300377651627395e-13,
 7: 7.741878983954696e-08,
 4: 0.1,
 6: 1.3189542556539747e-13,
 2: 5.644739300537776e-08,
 3: 7.513148009015778e-05}

In [543]:
assert get_label(posterior_prob) == (4, 0.1)

In [544]:
expected = {1: 6.300377651627395e-13,
 7: 7.741878983954696e-08,
 4: 0.1,
 6: 1.3189542556539747e-13,
 2: 5.644739300537776e-08,
 3: 7.513148009015778e-05}

In [545]:
assert posterior_prob == expected